<h2><b> Data Preprocessing </b></h2>

<h4> <b> Library Imports </b> </h4>

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

from config import *

<h4> <b> Import Data </b> </h4>

In [3]:
df = pd.read_csv('./../' + data_path)

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

<h4><b>Feature Engineering</b></h4>
<h5>Features such as <code>RowNumber</code>, <code>CustomerId</code>, and <code>Surname</code> do not contribute meaningfully to predicting whether a customer will exit the firm. Dropping these features can improve model performance and reduce noise.</h5>


In [6]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1) # columnwise

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


<h5>Features like <code>Geography</code> and <code>Gender</code> are categorical, so encoding is necessary. Since <code>Geography</code> has three distinct values, we use one-hot encoding. <code>Gender</code> is a binary categorical variable, so we apply label encoding.</h5>


In [8]:
# Encoding Gender Feature
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [9]:
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9115,612,Germany,0,36,2,130700.92,2,0,0,77592.80,0
3215,726,France,0,32,8,0.00,2,0,0,185075.63,0
220,726,France,0,31,9,114722.05,2,1,1,98178.57,0
8484,802,France,0,60,3,92887.06,1,1,0,39473.63,1
8759,748,France,1,66,8,0.00,1,1,1,163331.65,0


In [10]:
# Encoding Geography Feature
onehot_encoder = OneHotEncoder()
geography_feature = onehot_encoder.fit_transform(df[['Geography']])
geography_feature

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [11]:
geography_feature.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [12]:
geography_feature_names = onehot_encoder.get_feature_names_out(['Geography'])
geography_feature_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
geography_df = pd.DataFrame(geography_feature.toarray(), columns=geography_feature_names, dtype=int)
geography_df.head(10)

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1
5,0,0,1
6,1,0,0
7,0,1,0
8,1,0,0
9,1,0,0


In [14]:
df = df.drop(['Geography'], axis=1)
df = pd.concat([df, geography_df], axis=1)

In [15]:
df.head(10)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1
5,645,1,44,8,113755.78,2,1,0,149756.71,1,0,0,1
6,822,1,50,7,0.00,2,1,1,10062.80,0,1,0,0
7,376,0,29,4,115046.74,4,1,0,119346.88,1,0,1,0
8,501,1,44,4,142051.07,2,0,1,74940.50,0,1,0,0
9,684,1,27,2,134603.88,1,1,1,71725.73,0,1,0,0


<h5>We need to save the label encoder and one-hot encoder to ensure that the same encoding is applied when making predictions on new data.</h5>


In [16]:
with open('./../' + label_encoder_gender_path, 'wb') as file:
    pickle.dump(label_encoder, file)

with open('./../' + onehot_encoder_geography_path, 'wb') as file:
    pickle.dump(onehot_encoder, file)

<h5>We divide the dataset into independent features (X) and the dependent variable (y), followed by performing a train-test split to prepare the data for model training and evaluation.</h5>


In [17]:
X = df.drop(['Exited'], axis=1)
y = df['Exited']

In [18]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1,0,0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1,0,0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,0,0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0,1,0


In [19]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [20]:
# Scalling the data
scalar = StandardScaler()
X = scalar.fit_transform(X)
X

array([[-0.32622142, -1.09598752,  0.29351742, ...,  0.99720391,
        -0.57873591, -0.57380915],
       [-0.44003595, -1.09598752,  0.19816383, ..., -1.00280393,
        -0.57873591,  1.74273971],
       [-1.53679418, -1.09598752,  0.29351742, ...,  0.99720391,
        -0.57873591, -0.57380915],
       ...,
       [ 0.60498839, -1.09598752, -0.27860412, ...,  0.99720391,
        -0.57873591, -0.57380915],
       [ 1.25683526,  0.91241915,  0.29351742, ..., -1.00280393,
         1.72790383, -0.57380915],
       [ 1.46377078, -1.09598752, -1.04143285, ...,  0.99720391,
        -0.57873591, -0.57380915]])

In [21]:
with open('./../' + feature_scalar_path, 'wb') as file:
    pickle.dump(scalar, file)

In [22]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<h2><b> ANN Implementation </b></h2>

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [24]:
X_train.shape

(8000, 12)

In [25]:
# Build Model
model = Sequential([
    Dense(64, input_dim = X_train.shape[1], activation = 'relu'),   # Hidden layer 1
    Dense(32, activation = 'relu'),                                 # Hidden layer 2
    Dense(1, activation = 'sigmoid')                                # Output layer
])

c:\Users\Kaushik\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Model compilation
model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [28]:
# Logs 
log_dir = './logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [29]:
# Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [30]:
# Train the model
history = model.fit(
    X_train, 
    y_train, 
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7568 - loss: 0.5158 - val_accuracy: 0.8275 - val_loss: 0.3953
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8296 - loss: 0.4031 - val_accuracy: 0.8495 - val_loss: 0.3638
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8460 - loss: 0.3754 - val_accuracy: 0.8625 - val_loss: 0.3452
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8545 - loss: 0.3525 - val_accuracy: 0.8555 - val_loss: 0.3462
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8581 - loss: 0.3476 - val_accuracy: 0.8610 - val_loss: 0.3411
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8592 - loss: 0.3406 - val_accuracy: 0.8555 - val_loss: 0.3442
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8580 - loss: 0.3455 - val_accuracy: 0.8600 - val_loss: 0.3369
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8605 - loss: 0.3394 - val_accu

In [31]:
# save the model
model.save('./../models/ann_model.keras')

In [32]:
%load_ext tensorboard
%tensorboard --logdir logs/fit


Launching TensorBoard...